In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install mlpack
!pip install notebookjs

     |████████████████████████████████| 93.0 MB 64 kB/s 


In [4]:
from Topomap import TopoMap

import numpy as np
from sklearn import datasets
from notebookjs import execute_js
import glob
import pandas as pd
import random
import json

### Prepare Dataset

In [5]:
# digit
digit = datasets.load_digits()
X = digit.data
Y = digit.target
digit.data.shape

(1797, 64)

In [6]:
def get_data_by_dim(inputData, input_label, dim_array):
    posData = []
    topomap = TopoMap()
    dataNum = inputData.shape[0]

    filteredData = inputData[:,dim_array]
    
    outData = topomap.project(filteredData, len(dim_array))
    topore = np.array([[each.x, each.y] for each in outData]).T
    
    for i in range(dataNum):
        posData.append({'x':topore[0][i], 'y': topore[1][i], 'group': int(input_label[i])})
    
    return posData

### Draw scatter plot by D3

In [7]:
def draw_scatter(posData):
    d3_lib_url = "https://d3js.org/d3.v3.min.js"

    with open ("./scatterplot.js", "r") as f:
        scatterLib = f.read()

    execute_js(library_list=[d3_lib_url, scatterLib],
                main_function="scatterplot",
                data_dict=posData,
                )

In [8]:
jsonData = get_data_by_dim(X, Y, range(64))
draw_scatter(jsonData)

computing emst
placing points


### Prepare Dim Data

In [9]:
def dim_correlation(inputData):
    dataNum = inputData.shape[0]
    dimNum = inputData.shape[1]
    output = [[0 for col in range(dimNum)] for row in range(dimNum)]
    for i in range(dimNum):
        for j in range(dimNum):
            output[i][j] = correlation_metric(X[:,i],X[:,j])
    return output

def correlation_metric(x, y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [10]:
def get_dim_data(inputData):
  inputData = inputData.T;
  posData = []
  topomap = TopoMap()
  dataNum = inputData.shape[1]
  dimNum = inputData.shape[0]
  
  outData = topomap.project(inputData, dataNum);
  topore = np.array([[each.x, each.y] for each in outData]).T
  
  for i in range(dimNum):
      posData.append({'x':topore[0][i], 'y': topore[1][i], 'group': int(i)})
  
  return posData

In [13]:
def draw_dim_scatter(inputData):
    jsonData = get_dim_data(inputData);
    d3_lib_url = "https://d3js.org/d3.v3.min.js"

    with open ("./dimplot.js", "r") as f:
        dimScatterLib = f.read()
    with open ("./dimplot.css", "r") as f:
        dimScatterCss = f.read()

    execute_js(library_list=[d3_lib_url, dimScatterLib],
                main_function="dimplot",
                data_dict=jsonData,
                callbacks= {"get_data_update": get_data_update},
                css_list=[dimScatterCss]
                )

draw_dim_scatter(X)

computing emst
placing points


computing emst
placing points
computing emst
placing points


computing emst
placing points


computing emst
placing points


In [12]:
def get_data_update(callback_data):
  global temp
  selected_dim = []
  callback_array = callback_data['datafiltered']
  for obj in callback_array:
    selected_dim.append(obj['group'])
  jsonData = get_data_by_dim(X, Y, selected_dim)
  draw_scatter(jsonData)
  